# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [17]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score,  r2_score,  confusion_matrix

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [3]:
#your code here
spaceship = spaceship.dropna().reset_index(drop=True)
spaceship['Cabin'] = spaceship['Cabin'].str.extract(r'([A-Z])')
spaceship.drop(columns=['PassengerId', "Name"], inplace=True)
spaceship = pd.get_dummies(spaceship, columns=['HomePlanet', 'Cabin', 'Destination'], drop_first=True)
features = spaceship.drop(columns='Transported')
target = spaceship["Transported"]

In [4]:
#your code here
scaler_minmax = MinMaxScaler()
features_scaled_x = scaler_minmax.fit_transform(features)
features_scaled = pd.DataFrame(features_scaled_x, columns=features.columns)

In [10]:
#your code here
#your code here
features_train, features_test, target_train, target_test = train_test_split(features_scaled, target, test_size=0.2, random_state=0)


print('features_train shape is:',features_train.shape)
print('features_test shape is:', target_train.shape)
print('features_test shape is:', features_test.shape)
print('target_train shape is:', target_test.shape)

features_train shape is: (5284, 19)
features_test shape is: (5284,)
features_test shape is: (1322, 19)
target_train shape is: (1322,)


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [8]:
#your code here
#your code here
forest = RandomForestClassifier(n_estimators=100,
                             max_depth=20)

In [11]:
forest.fit(features_train, target_train)

RandomForestClassifier(max_depth=20)

- Evaluate your model

In [18]:
#your code here
pred = forest.predict(features_test)

print("R2 score", forest.score(features_test, target_test))
print("Accuracy", accuracy_score(target_test, pred))
print("Recall", recall_score(target_test, pred))
print("Precision", precision_score(target_test, pred, average='binary')) # average = 'binary' is the default

R2 score 0.7859304084720121
Accuracy 0.7859304084720121
Recall 0.7776096822995462
Precision 0.7907692307692308


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [23]:
#your code here
grid = {"n_estimators": [50, 100, 200,500],
        "max_leaf_nodes": [250, 500, 1000, None],
        "max_depth":[10,30,50]}

- Run Grid Search

In [24]:
rad_f = RandomForestClassifier()

In [25]:
model = GridSearchCV(estimator = rad_f, param_grid = grid, cv=5) # cv crossvalidation 5 é a quantidade de vezes para tentar


In [26]:
model.fit(features_train, target_train)


GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [10, 30, 50],
                         'max_leaf_nodes': [250, 500, 1000, None],
                         'n_estimators': [50, 100, 200, 500]})

In [27]:
model.best_params_


{'max_depth': 10, 'max_leaf_nodes': 250, 'n_estimators': 500}

In [28]:
best_model = model.best_estimator_


- Evaluate your model

In [38]:

pred = best_model.predict(features_test)

# print("MAE", mean_absolute_error(pred, target_test))
# print("RMSE", mean_squared_error(pred, target_test, squared=False))
print("R2 score", best_model.score(features_test, target_test))

# print("R2 score", forest.score(features_test, target_test))
print("Accuracy", accuracy_score(target_test, pred))
print("Recall", recall_score(target_test, pred))
print("Precision", precision_score(target_test, pred, average='binary')) # average = 'binary' is the default

R2 score 0.7851739788199698
Accuracy 0.7851739788199698
Recall 0.8018154311649016
Precision 0.7759882869692533


In [39]:
grid = {"n_estimators": [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
        "max_leaf_nodes": [int(x) for x in np.linspace(start = 500, stop = 3000, num = 10)],
        "max_depth":[int(x) for x in np.linspace(10, 110, num = 11)]}

In [40]:
rad_f = RandomForestClassifier()

model = RandomizedSearchCV(estimator = rad_f, param_distributions = grid, n_iter = 10, cv = 5, n_jobs = -1)

In [41]:
model.fit(features_train, target_train)


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110],
                                        'max_leaf_nodes': [500, 777, 1055, 1333,
                                                           1611, 1888, 2166,
                                                           2444, 2722, 3000],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]})

In [42]:

model.best_params_


{'n_estimators': 200, 'max_leaf_nodes': 1611, 'max_depth': 10}

In [43]:
best_model = model.best_estimator_


- Evaluate our model

In [46]:
pred = best_model.predict(features_test)

# print("MAE", mean_absolute_error(pred, target_test))
# print("RMSE", mean_squared_error(pred, target_test, squared=False))
print("R2 score", best_model.score(features_test, target_test))
print("Accuracy", accuracy_score(target_test, pred))
print("Recall", recall_score(target_test, pred))
print("Precision", precision_score(target_test, pred, average='binary')) # average = 'binary' is the default

R2 score 0.7881996974281392
Accuracy 0.7881996974281392
Recall 0.7987897125567323
Precision 0.7822222222222223
